In [ ]:
# Importing necessary libraries
from matplotlib import legend
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import os
import numpy as np
from icecream import ic
from tqdm import tqdm
from itertools import chain
import logging
import os
import cProfile
import pstats
#from dataloader import *
from get_data import *
from torch.utils.data import Dataset, DataLoader

In [ ]:
visualise(get_data(path = "save_data_test3.csv", timesteps_from_data=0, skip_steps_start = 0, skip_steps_end = 0, 
                   drop_half_timesteps = False, normalise_s_w=False, rescale_p=False, num_inits=100), num_inits=10, set_ylim=True)

In [ ]:
#custom dataclass

#This custom dataclass moves a sliding window over a timeseries and returns
# inp(0:t) and label(t+1)
#if values are missing the last value is used for padding sind 
#Structure of the data:
# (number of timeseries // timesteps // features)
# if index+window_size > timesteps  >>> padding

import torch.utils
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data, window_size, future=1):

        self.data = data
        self.ws = window_size
        self.future = future

    def __len__(self):
        return self.data.size(0)*self.data.size(1) - (self.ws + 1) - (self.future-1)

    def __getitem__(self, idx):

        j = int(idx/self.data.size(1))  

        k = int((idx + self.ws + (self.future-1)) / self.data.size(1))

        m = (idx + self.ws) - k * self.data.size(1)

        index = idx % self.data.size(1)

        if j < k :
            print(m)
            if m < 0: 
                inp = self.data[j, index : index + self.ws, :]
            else: 
                inp=torch.cat((self.data[j, index : self.data.size(1) , :],
                          self.data[j, self.data.size(1) - 1, :].repeat(m, 1)))
                
            if self.future>1:
                label = self.data[j, self.data.size(1) - 1, :].repeat(self.future, 1)        
            else:
                label = self.data[j, self.data.size(1) - 1, :]
                
            
        else:

            inp = self.data[j, index : index + self.ws, :]

            if self.future>1:
                label = self.data[j, index + self.ws : index + self.ws + self.future  , :]
            else:
                label = self.data[j, index + self.ws, :]

        return inp, label

    def get_all_data(self):
        return self.data

In [33]:
input_data = get_data(path = "save_data_test3.csv", 
                        timesteps_from_data=0, 
                        skip_steps_start = 0,
                        skip_steps_end = 0, 
                        drop_half_timesteps = False,
                        normalise_s_w=True,
                        rescale_p=False,
                        num_inits=3)

cut_off_timesteps = 1080

#Split data into train and test sets
percentage_of_data = 0.9
window_size = 4
batch_size = 2

num_of_inits_train = int(len(input_data)*percentage_of_data)
train_inits = np.random.randint(0,len(input_data), num_of_inits_train)
train_inits = np.unique(train_inits)
test_inits = np.array([x for x in range(len(input_data)) if x not in train_inits])

# make sure we really get the specified percentage of training data..
if percentage_of_data < 0.99: 
        while len(train_inits) < num_of_inits_train:
            i = np.random.randint(0,len(test_inits),1)[0]
            train_inits = np.append(train_inits,test_inits[i])
            test_inits = np.delete(test_inits, i)

train_data = input_data[train_inits,:input_data.size(dim=1)-cut_off_timesteps,:]
test_data = input_data[test_inits,:,:]

data_set  = CustomDataset(train_data, window_size=window_size, future=3)
train_dataloader = DataLoader(data_set, batch_size=batch_size, pin_memory=True, drop_last=True)

In [34]:
print(train_data.size())
for k, (x,y) in enumerate(train_dataloader):
    #print(x)
    #print("label:",y)
    #print("timestep mit label simma bei t = ", k, ":", k+(window_size+2))
    print("-----------------------------")
    print(k)

    print(x)
    print(y)
    print(x.size())
    print(y.size())

torch.Size([2, 20, 3])
-----------------------------
0
tensor([[[ 0.9219,  1.8464,  0.1363],
         [ 0.9202,  1.8460,  0.0838],
         [ 0.9184,  1.8450,  0.0420],
         [ 0.9167,  1.8433,  0.0039]],

        [[ 0.9202,  1.8460,  0.0838],
         [ 0.9184,  1.8450,  0.0420],
         [ 0.9167,  1.8433,  0.0039],
         [ 0.9150,  1.8412, -0.0327]]], dtype=torch.float64)
tensor([[[ 0.9150,  1.8412, -0.0327],
         [ 0.9133,  1.8385, -0.0683],
         [ 0.9116,  1.8354, -0.1031]],

        [[ 0.9133,  1.8385, -0.0683],
         [ 0.9116,  1.8354, -0.1031],
         [ 0.9099,  1.8317, -0.1372]]], dtype=torch.float64)
torch.Size([2, 4, 3])
torch.Size([2, 3, 3])
-----------------------------
1
tensor([[[ 0.9184,  1.8450,  0.0420],
         [ 0.9167,  1.8433,  0.0039],
         [ 0.9150,  1.8412, -0.0327],
         [ 0.9133,  1.8385, -0.0683]],

        [[ 0.9167,  1.8433,  0.0039],
         [ 0.9150,  1.8412, -0.0327],
         [ 0.9133,  1.8385, -0.0683],
         [ 0.9116, 

tensor([[[1.1196, 0.6979, 0.1911],
         [1.1345, 0.6980, 0.2330],
         [1.1494, 0.6984, 0.2672],
         [1.1643, 0.6990, 0.2993]],

        [[1.1345, 0.6980, 0.2330],
         [1.1494, 0.6984, 0.2672],
         [1.1643, 0.6990, 0.2993],
         [1.1792, 0.6998, 0.3319]]], dtype=torch.float64)
tensor([[[1.1792, 0.6998, 0.3319],
         [1.1941, 0.7008, 0.3658],
         [1.2090, 0.7020, 0.4017]],

        [[1.1941, 0.7008, 0.3658],
         [1.2090, 0.7020, 0.4017],
         [1.2239, 0.7035, 0.4395]]], dtype=torch.float64)
torch.Size([2, 4, 3])
torch.Size([2, 3, 3])
-----------------------------
11
tensor([[[1.1494, 0.6984, 0.2672],
         [1.1643, 0.6990, 0.2993],
         [1.1792, 0.6998, 0.3319],
         [1.1941, 0.7008, 0.3658]],

        [[1.1643, 0.6990, 0.2993],
         [1.1792, 0.6998, 0.3319],
         [1.1941, 0.7008, 0.3658],
         [1.2090, 0.7020, 0.4017]]], dtype=torch.float64)
tensor([[[1.2090, 0.7020, 0.4017],
         [1.2239, 0.7035, 0.4395],
        

In [ ]:
ws= 3
n = 2

data = get_data(path = "save_data_test3.csv", timesteps_from_data=0, skip_steps_start = 0, skip_steps_end = 1000, drop_half_timesteps = True, normalise_s_w=False, rescale_p=False, num_inits=n)
data_set = CustomDataset(data, window_size=ws, future=1)
dataloader = DataLoader(data_set, batch_size=2)

time = np.linspace(0,data.size(dim=1)*n,data.size(dim=1)*n)
print(data.size())
for i, (inp, label) in enumerate(dataloader):
    #print("input" , inp)
    #print("label",label)
    print(i, label.size())

In [ ]:
a,b = next(iter(dataloader))
c, d = next(iter(dataloader))

print(a,b)
print(c,d)

In [ ]:


import matplotlib.pyplot as plt
import numpy as np

# Generate some sample data
x = np.arange(10)  # Extend the range of x-values
y1 = np.array([1, 2, 3, 4])
y2 = np.array([2, 3, 4, 5])
y3 = np.array([3, 4, 5, 6])

# Plot each array with offset
plt.plot(x[:4], y1, label='Array 1')
plt.plot(x[1:5], y2, label='Array 2')
plt.plot(x[2:6], y3, label='Array 3')
plt.xlim(0,10)
# Add labels and legend
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Arrays Offset Plot')
plt.legend()

# Show plot
plt.grid(True)
plt.show()